In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import psycopg2

In [2]:
# Variablen
conn_Ok = True


In [3]:
# Check if connection to DB possible
try:
    connection = psycopg2.connect(
        dbname='PSQL_ADSFS2023Gruppe15',
        user='ADSFS2023Gruppe15',
        password='ADS_FS_2023_G15!?',
        host='localhost',
        port='5432'
    )

    cursor = connection.cursor()

except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
        conn_Ok = False

In [4]:
def get_team_id(team_name):

    if team_name == 'Bor. Mönchengladbach':
        team_name = 'Borussia Mönchengladbach'
    elif team_name == '1899 Hoffenheim':
        team_name = 'TSG 1899 Hoffenheim'
    elif team_name == 'SV Werder Bremen':
        team_name = 'Werder Bremen'
        
    # Vergleicht Team Namen und sucht nach ID
    cursor.execute("SELECT team_id FROM bundesliga_mannschaften WHERE mannschaft = %s", (team_name,))
    result = cursor.fetchone()[0]

    if result:
        return result
    else:
        print(f'Keine Übereinstimmung für {team_name} gefunden')


In [5]:
if conn_Ok:
    # Anzahl Pässe für die spielTage im angegebenen Range
    for matchday in range(1, 35):
        url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-paesse/'

        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Suche nach allen Mannschaftsnamen und Passquoten
            team_elements = soup.find_all('td', class_='team-name team-name-list')
            pass_complete_elements = soup.find_all('td', class_='team_stats-passes_complete team_stats-passes_complete-list')
            pass_failed_elements = soup.find_all('td', class_='team_stats-passes_failed team_stats-passes_failed-list')
            pass_total_elements = soup.find_all('td', class_='team_stats-passes team_stats-passes-list')
            pass_percentage_elements = soup.find_all('td', class_='team_stats-passes_complete_percentage team_stats-passes_complete_percentage-format')

            # Iteriere über die gefundenen Elemente und extrahiere die gewünschten Informationen
            for team_element,pass_complete_element, pass_failed_element,pass_total_element, pass_percentage_element in zip(team_elements,pass_complete_elements, pass_failed_elements, pass_total_elements, pass_percentage_elements):
                team_name = team_element.text.strip()
                pass_complete = pass_complete_element.text.strip()
                pass_failed = pass_failed_element.text.strip()
                pass_total = pass_total_element.text.strip()
                pass_percentage = pass_percentage_element.text.strip()

                # Speichere die Daten in der PostgreSQL-Datenbank
                insert_data_query = '''
                INSERT INTO bundesliga_pass_stats (matchday, team_id, pass_complete, pass_failed, pass_total, pass_percentage) 
                VALUES (%s, %s, %s,%s, %s, %s);
                '''

                team_id = get_team_id(team_name)

                cursor.execute(insert_data_query, (matchday, team_id, pass_complete, pass_failed, pass_total, float(pass_percentage.replace(',', '.'))))

                connection.commit()
        else:
            print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday}')

In [6]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-torschuesse/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        shots_elements = soup.find_all('td', class_='team_stats-shots team_stats-shots-list')

        for team_element, shots_element in zip(team_elements, shots_elements):
            team_name = team_element.text.strip()
            shots_total = shots_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_shots_stats (matchday, team_id, shots_total)
                VALUES (%s, %s, %s);
            '''
            team_id = get_team_id(team_name)

            cursor.execute(insert_data_query, (matchday, team_id, shots_total))

            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Torschüsse')

In [7]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-zweikaempfe/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        duels_elements = soup.find_all('td', class_='team_stats-duels team_stats-duels-list')
        duels_won_elements = soup.find_all('td', class_='team_stats-duels_won team_stats-duels_won-list')

        for team_element, duels_element, duels_won_element in zip(team_elements, duels_elements, duels_won_elements):
            team_name = team_element.text.strip()
            duels_total = duels_element.text.strip()
            duels_won = duels_won_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_duels_stats (matchday, team_id, duels_total, duels_won)
                VALUES (%s, %s, %s, %s);
                '''
            team_id =get_team_id(team_name)
        
            cursor.execute(insert_data_query, (matchday, team_id, duels_total, duels_won))
                        
            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Zweikämpfe')